In [2]:
import numpy as np
import scipy.interpolate as interpolate

In [43]:
env = ['duck_lift', 'banana_pass1', 'hammer_strike', 'mug_drink3', 'toothpaste_lift'][-1]
# data = np.load(f'{env}.npz', allow_pickle=True,)
# entries = data.files
# print(entries)
def load_motion(motion_file):
    motion_file = np.load(motion_file, allow_pickle=True)
    reference_motion =  {k:v for k, v in motion_file.items()}
    reference_motion['s_0'] = reference_motion['s_0'][()]
    return reference_motion
data = load_motion(f'{env}.npz')
data.keys()

dict_keys(['s_0', 'object_translation', 'object_orientation', 'length', 'SIM_SUBSTEPS', 'DATA_SUBSTEPS'])

In [44]:
def data_analysis(data):
    print('range of data: ', np.min(data, axis=0), np.max(data, axis=0))
    print('start and end: ', data[0], data[-1])

In [45]:
print(data['object_translation'].shape)
trans_data = data['object_translation']  # x, y, z
data_analysis(trans_data)

(44, 3)
range of data:  [-0.00423752 -0.02918028  0.01676679] [0.02003396 0.00393653 0.23768179]
start and end:  [-0.00238724  0.00120428  0.01676679] [ 0.01904994 -0.02066231  0.01732265]


In [46]:
print(data['object_orientation'].shape)
ori_data = data['object_orientation']
data_analysis(ori_data)

(44, 4)
range of data:  [ 0.6711423  -0.2323463  -0.66860282 -0.30602647] [ 0.7219608  -0.16035895 -0.60405616 -0.2188484 ]
start and end:  [ 0.67822219 -0.2323463  -0.65430795 -0.24064697] [ 0.68366748 -0.22081987 -0.65895671 -0.2227407 ]


In [47]:
def interpolate_data(data, points=None, initial_point=None, random_sample=5):
    if points is None:
        points = np.random.uniform(np.min(data), np.max(data), size=random_sample)
    if initial_point is not None: # set the same initial point as original traj
        points[0] = initial_point
    else:
        points[0] = data[0]
    len_points = points.shape[0]
    f = interpolate.interp1d(np.arange(len_points), points, kind='quadratic', fill_value="extrapolate")  # ‘linear’, ‘nearest’, ‘nearest-up’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’, or ‘next’. 

    x_new = np.arange(0, len_points-1, (len_points-1)/data.shape[0])
    y_new = f(x_new)
    y_new[-1] = points[-1]
    return y_new

In [48]:
import copy
new_trans_data = []
new_points = np.array([copy.deepcopy(data['object_translation'][0]) for _ in range(3)])
new_points[1][2] += 0.4 # z
new_points[2][1] += 0.2 # y
new_points[2][2] += -0.1 # z

print(new_points)
for d, nd in zip(trans_data.T, new_points.T):
    new_trans_data.append(interpolate_data(d, nd))
new_trans_data = np.array(new_trans_data).T
print(new_trans_data.shape)

# new_ori_data = []
# for d in ori_data.T:
#     new_ori_data.append(interpolate_data(d))
# new_ori_data = np.array(new_ori_data).T
# print(new_ori_data.shape)
new_ori_data = ori_data

[[-0.00238724  0.00120428  0.01676679]
 [-0.00238724  0.00120428  0.41676679]
 [-0.00238724  0.20120428 -0.08323321]]
(44, 3)


In [49]:
import copy
save_dir = f'specified_trajs'
os.makedirs(save_dir, exist_ok=True)
new_traj = copy.copy(dict(data))

new_traj['object_translation'] = new_trans_data
new_traj['object_orientation'] = new_ori_data
print(new_traj.keys())
np.savez(save_dir+f'/{env}', **new_traj)  # save a dict as npz

dict_keys(['s_0', 'object_translation', 'object_orientation', 'length', 'SIM_SUBSTEPS', 'DATA_SUBSTEPS'])
